In [10]:
!python -m spacy download en


     ---------------------------------------- 12.8/12.8 MB 1.8 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2023-04-05 12:13:08.620867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-05 12:13:08.620914: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-05 12:13:11.852416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-04-05 12:13:11.852450: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-05 12:13:11.855399: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-38ONH9K
2023-04-05 12:13:11.855469: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-38ONH9K
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded mo

In [11]:
!pip install spacy[en]


In [12]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2023-04-05 12:13:56.088971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-05 12:13:56.089019: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-05 12:13:59.238088: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-04-05 12:13:59.238116: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-05 12:13:59.240976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-38ONH9K
2023-04-05 12:13:59.241054: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-38ONH9K
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded mo

In [3]:
dataset = load_dataset(
    "tner/bionlp2004", 
    cache_dir='./data_cache'
)

print(f'The dataset is a dictionary with {len(dataset)} splits: \n\n{dataset}')

NameError: name 'load_dataset' is not defined

In [1]:
import spacy
from pathlib import Path

nlp = spacy.load("en_core_web_sm")
language = nlp.meta["lang"]

data_path = Path(spacy.__file__).parent / "data"
lookups_path = data_path / language / f"{language}_lookups.json"

print(lookups_path)


C:\Users\abdul\anaconda3\envs\enr\lib\site-packages\spacy\data\en\en_lookups.json


In [2]:
import spacy
import random
from spacy.util import minibatch, compounding


# Define the training data
train_data = [
    ("Elon Musk is the CEO of SpaceX and Tesla.", {"entities": [(0, 8, "PERSON"), (23, 28, "ORG"), (33, 38, "ORG")]}),
    ("Steve Jobs was the CEO of Apple.", {"entities": [(0, 10, "PERSON"), (24, 29, "ORG")]}),
    ("Barack Obama was the 44th President of the United States.", {"entities": [(0, 12, "PERSON"), (34, 50, "GPE")]}),
    ("Satya Nadella is the CEO of Microsoft.", {"entities": [(0, 12, "PERSON"), (23, 33, "ORG")]}),
    ("Amazon is the world's largest online retailer.", {"entities": [(0, 6, "ORG")]}),
    ("Mark Zuckerberg is the CEO of Facebook.", {"entities": [(0, 14, "PERSON"), (23, 31, "ORG")]}),
]

# Define the function to train the model
def train_model(train_data, iterations):

    # Load the spaCy model with the path to the lookup tables
    nlp = spacy.load("en_core_web_sm", exclude=["parser", "ner"], lookups=lookups_path)

    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")

    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for i in range(iterations):
            random.shuffle(train_data)
            losses = {}
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)
    return nlp


# Train the model
model = train_model(train_data, iterations=100)

# Test the model
text = "Jeff Bezos is the founder of Amazon."
doc = model(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])


C:\Users\abdul\anaconda3\envs\enr\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Elon Musk is the CEO of SpaceX and Tesla." with entities "[(0, 8, 'PERSON'), (23, 28, 'ORG'), (33, 38, 'ORG'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\abdul\anaconda3\envs\enr\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Steve Jobs was the CEO of Apple." with entities "[(0, 10, 'PERSON'), (24, 29, 'ORG')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\abdul\anaconda3\envs\enr\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities c

Losses {'ner': 8.493480837100172}
Losses {'ner': 8.25667298793175}
Losses {'ner': 4.598021725868421}
Losses {'ner': 2.9665479305567235}
Losses {'ner': 5.917030490791238}
Losses {'ner': 5.640648760218085}
Losses {'ner': 6.773055590106651}
Losses {'ner': 1.9615919114081084}
Losses {'ner': 4.219486742215118}
Losses {'ner': 5.376346936847625}
Losses {'ner': 0.11255781556617678}
Losses {'ner': 1.3494327464794034}
Losses {'ner': 0.04187934193344675}
Losses {'ner': 2.7366248860900133}
Losses {'ner': 0.19335129249938837}
Losses {'ner': 2.5105282781453635}
Losses {'ner': 0.09814465328435618}
Losses {'ner': 0.004293845070271283}
Losses {'ner': 3.1361011982388334}
Losses {'ner': 1.2624570904219983}
Losses {'ner': 1.9344839841398287}
Losses {'ner': 1.1169383715815366}
Losses {'ner': 1.9873940701885193}
Losses {'ner': 0.951446766934929}
Losses {'ner': 0.0017042636317348943}
Losses {'ner': 0.08521360092290241}
Losses {'ner': 3.390597311453636}
Losses {'ner': 0.41348719631765446}
Losses {'ner': 0.044

In [1]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the financial_phrasebank dataset
from datasets import load_dataset

import numpy as np

C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import spacy

In [3]:
dataset = load_dataset(
    "tner/bionlp2004", 
    cache_dir='./data_cache'
)

print(f'The dataset is a dictionary with {len(dataset)} splits: \n\n{dataset}')

Found cached dataset bionlp2004 (F:/ICRL/projects/55/Sentiment classification/data_cache/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904)
100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 500.33it/s]

The dataset is a dictionary with 3 splits: 

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 16619
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1927
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3856
    })
})


In [4]:
train_sentences_ner = [item['tokens'] for item in dataset['train']]
train_sentences = [' '.join(sentence) for sentence in train_sentences_ner]
train_labels_ner = [[str(tag) for tag in item['tags']] for item in dataset['train']]

val_sentences_ner = [item['tokens'] for item in dataset['validation']]
val_sentences = [' '.join(sentence) for sentence in val_sentences_ner]
val_labels_ner = [[str(tag) for tag in item['tags']] for item in dataset['validation']]

test_sentences_ner = [item['tokens'] for item in dataset['test']]
test_sentences = [' '.join(sentence) for sentence in test_sentences_ner]
test_labels_ner = [[str(tag) for tag in item['tags']] for item in dataset['test']]

In [37]:
nlp = spacy.load("en_core_web_sm")

In [29]:
corpus = []
text = train_sentences
doc = nlp(text[0])

for sent in doc.sents:
    print(sent)
    corpus.append(sent.text)

Since HUVECs released superoxide anions in response to TNF , and H2O2 induces VCAM-1 , PDTC may act as a radical scavenger .


In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = "Treblinka is a small village in Poland. Wikipedia notes that Treblinka is not large."
corpus = []

doc = nlp(text)
for sent in doc.sents:
    corpus.append(sent.text)

# nlp = spacy.blank("en")

# ruler = nlp.add_pipe("entity_ruler")

# patterns = [
#                 {"label": "GPE", "pattern": "Treblinka"}
#             ]

# ruler.add_patterns(patterns)

TRAIN_DATA = []
for sentence in corpus:
    doc = nlp(sentence)
    entities = []

    for ent in doc.ents:
        entities.append([ent.start_char, ent.end_char, ent.label_])
    TRAIN_DATA.append([sentence, {"entities": entities}])

print (TRAIN_DATA)

[['Treblinka is a small village in Poland.', {'entities': [[32, 38, 'GPE']]}], ['Wikipedia notes that Treblinka is not large.', {'entities': [[0, 9, 'ORG']]}]]


In [6]:
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)
    
convert("en", TRAIN_DATA, "train.spacy")

In [8]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-04-05 11:54:56.481039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-05 11:54:56.481085: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-05 11:54:59.741628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-04-05 11:54:59.741654: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-05 11:54:59.744742: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-38ONH9K
2023-04-05 11:54:59.744815: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-38ONH9K
C:\Users\abdul\anaconda3\envs\tf2x\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded mo

In [33]:
for sentence in corpus:
    doc = nlp(sentence)
    print(doc)
    entities = []
    TRAIN_DATA = []
    for ent in doc.ents:
        print(ent.start)
        print(ent.end)
        print(ent.label_)
        entities.append([ent.start_char, ent.end_char, ent.label_])
    TRAIN_DATA.append([sentence, {"entities": entities}])

Since HUVECs released superoxide anions in response to TNF , and H2O2 induces VCAM-1 , PDTC may act as a radical scavenger .
8
9
ORG
11
12
ORG
15
16
ORG
